# Running ImageJ via Python

The idea behind the code below is to be able to run my ImageJ code from Python. Not sure if it will work.

In [ ]:
import imagej
import scyjava as sj

# initialize imagej
ij = imagej.init('sc.fiji:fiji:2.3.1', mode='interactive')

### Grouping all of the csv files

The code below should be run after you run the Endplate-analysis.ijm script.

In [1]:
import os
import numpy as np
import pandas as pd

def group_func(s):
    """Add Group Info based on Mouse #"""
    if int(s) in [7488, 8834, 8835, 7482, 7485, 8831]:
        group = "Crush"
    elif int(s) in [8593, 8836, 8837, 8591, 8832, 8833]:
        group = "Sham"
    elif int(s) in [8594, 8595, 7591, 7592, 8592]:
        group = "Tetanus"
    elif int(s) in [123, 144, 151, 125, 135, 152]:
        group = "Tetanus-Crush"
    else:
        group = "Other"
    
    return group

def percent_func(s):
    '''Adjusting values that are over 100% down to 100%'''
    if s > 1:
        val = 100
    else:
        val = s*100
    return val

def muscle_group(s):
    '''Adjusting the groups to be the same'''
    if s in ['GAS','MG','LG']:
        val = 'LG-MG'
    else:
        val = s
    return val

In [2]:
# Grabbing the files
path = r'G:\Tetanus\NMJ Data\raw images\Completed\Analysis_Results'
dir_list = os.listdir(path)

filelist = [file for file in dir_list if file.endswith(".csv")]

In [3]:
# Creating the dataframe
df = pd.DataFrame()
for file in filelist:
    titlesplit = file.split('_')
    
    df1 = pd.read_csv(os.path.join(path,file),index_col=0)
    df1.dropna(axis=0, how='all', inplace=True)
    
    titles = {'Mouse':[titlesplit[0] for i in range(len(df1))],
              'Muscle':[titlesplit[1][:-1].upper() for i in range(len(df1))]}
    
    df1=df1.assign(**titles)
    
    df = pd.concat([df,df1], ignore_index=True)

df["Group"] = df["Mouse"].apply(group_func)
df["Muscle"] = df["Muscle"].apply(muscle_group)

In [4]:
df.head()

,varea1,varea2,barea1,barea2,marea1,marea2,vcov11,vcov12,vcov22,mcov11,mcov22,Mouse,Muscle,Group
0,2.560,0.522,15.126,9.957,2.181,0.284,0.169,0.257,0.052,0.144,0.029,123,TA,Tetanus-Crush
1,16.850,5.576,35.907,32.966,16.115,5.331,0.469,0.511,0.169,0.449,0.162,123,TA,Tetanus-Crush
2,15.589,7.711,34.992,32.007,15.174,7.629,0.445,0.487,0.241,0.434,0.238,123,TA,Tetanus-Crush
3,1.667,0.000,21.167,11.500,1.333,0.000,0.079,0.145,0.000,0.063,0.000,123,TA,Tetanus-Crush
4,12.146,5.501,32.244,28.649,11.710,5.011,0.377,0.424,0.192,0.363,0.175,123,TA,Tetanus-Crush


In [5]:
for title in ["vcov11", "vcov12", "vcov22", "mcov11", "mcov22"]:
    df[title] = df[title].apply(percent_func)

df.head()

,varea1,varea2,barea1,barea2,marea1,marea2,vcov11,vcov12,vcov22,mcov11,mcov22,Mouse,Muscle,Group
0,2.560,0.522,15.126,9.957,2.181,0.284,16.9,25.7,5.2,14.4,2.9,123,TA,Tetanus-Crush
1,16.850,5.576,35.907,32.966,16.115,5.331,46.9,51.1,16.9,44.9,16.2,123,TA,Tetanus-Crush
2,15.589,7.711,34.992,32.007,15.174,7.629,44.5,48.7,24.1,43.4,23.8,123,TA,Tetanus-Crush
3,1.667,0.000,21.167,11.500,1.333,0.000,7.9,14.5,0.0,6.3,0.0,123,TA,Tetanus-Crush
4,12.146,5.501,32.244,28.649,11.710,5.011,37.7,42.4,19.2,36.3,17.5,123,TA,Tetanus-Crush


In [6]:
grouped = df.groupby("Group")
for name, group in grouped:
    group.sort_values(["Muscle","Mouse"]).to_csv(os.path.join(path,name+"_Data.csv"), index=False)

In [7]:
mus_ave = df.groupby(["Group", "Muscle"]).mean()
mus_ave.to_csv(os.path.join(path,"Muscle_Average.csv"))
mus_ave

varea1     varea2     barea1     barea2     marea1  \
Group         Muscle                                                          
Crush         LG-MG   19.752623  10.443495  35.221532  29.735054  15.805545   
              SOL     27.024283  18.276867  37.764814  30.368221  15.913735   
              TA      14.840971   6.838175  33.365102  27.253500  12.164029   
Other         TA       9.963803   3.696224  35.440474  28.113382   8.548526   
Sham          LG-MG   15.258222   7.298429  32.394831  27.225620  11.906054   
              SOL     24.446101  15.682067  38.635437  32.425412  20.495647   
              TA      13.239682   6.058923  31.290974  26.140639  10.526582   
Tetanus       LG-MG   10.197049   4.113867  29.676302  24.489938   8.040104   
              TA       9.830500   3.826140  29.275073  24.311080   7.068107   
Tetanus-Crush LG-MG   13.665667   6.906909  32.751065  28.184204  10.538828   
              TA      13.628024   4.925313  31.703531  25.876218  11.176326   

                         marea2     vcov11     vcov12     vcov22     mcov11  \
Group         Muscle                                                          
Crush         LG-MG    8.046855  54.943098  63.772054  33.957912  43.750168   
              SOL      8.215301  70.011504  84.248673  58.808850  40.349558   
              TA       5.425879  42.392718  51.785922  23.372816  34.607282   
Other         TA       2.956250  27.017105  34.172368  12.792105  22.769737   
Sham          LG-MG    5.491651  44.964638  52.946053  25.126480  34.968257   
              SOL     11.658966  62.742857  73.576471  48.052101  52.024370   
              TA       4.429926  41.470739  49.185795  22.801989  32.984375   
Tetanus       LG-MG    2.761396  34.697727  41.568831  17.195130  27.560714   
              TA       2.411487  32.499667  38.832000  14.971646  23.109333   
Tetanus-Crush LG-MG    5.405909  40.391935  45.394624  22.125806  30.886559   
              TA       3.854606  41.771159  50.269542  18.287601  34.351482   

                         mcov22  
Group         Muscle             
Crush         LG-MG   25.966330  
              SOL     25.299115  
              TA      18.143689  
Other         TA       9.797368  
Sham          LG-MG   18.423684  
              SOL     33.967227  
              TA      16.410227  
Tetanus       LG-MG   11.407468  
              TA       9.228000  
Tetanus-Crush LG-MG   16.886559  
              TA      14.056334

In [8]:
mouse_ave = df.groupby(["Group", "Muscle", "Mouse"]).mean()
mouse_ave.to_csv(os.path.join(path,"Mouse_Average.csv"))
mouse_ave

varea1     varea2     barea1     barea2  \
Group         Muscle Mouse                                               
Crush         LG-MG  7482   16.883286   6.897524  36.488524  29.760571   
                     7485   10.372630   3.448111  36.553259  31.013259   
                     7488   20.893675   9.254925  39.513500  33.439900   
                     8831   23.508911  14.051987  34.205190  29.172190   
                     8834   21.354581  12.146070  35.793686  30.140105   
                     8835   15.965318   7.702386  30.604341  25.789386   
              SOL    8831   25.996676  17.980632  36.473618  29.222059   
                     8834   28.577111  18.724511  39.715956  32.100200   
              TA     7482   14.663698   6.098721  33.585721  27.552674   
                     7485   14.368880   5.721720  34.308520  29.462920   
                     7488   14.521333   6.498091  31.405485  26.152455   
                     8831    9.166810   2.877095  30.828857  22.925286   
                     8834   21.629295  12.928864  34.739386  28.679205   
                     8835   11.102075   3.991250  33.974800  27.163400   
Other         TA     173     9.963803   3.696224  35.440474  28.113382   
Sham          LG-MG  8591   17.480722   9.067835  32.212937  27.908620   
                     8593   18.123167   8.276956  36.418254  29.686246   
                     8832    9.541723   4.008492  32.232323  27.160646   
                     8833    8.564131   3.117720  25.681299  22.112206   
                     8836   17.882481   9.198245  32.424991  27.318377   
                     8837   17.502657   8.820080  34.448942  28.736993   
              SOL    8591   24.042789  14.981395  41.342395  33.718158   
                     8836   20.420784  14.074980  34.646255  29.054353   
                     8837   31.800000  19.301633  41.988233  36.518733   
              TA     8591   12.899825   5.833952  27.346714  22.399619   
                     8593   15.164905   7.205635  33.299238  27.472238   
                     8833   12.408541   4.337885  36.198328  29.936672   
                     8836   11.684767   5.488977  28.752692  24.243985   
                     8837   18.165469   9.893812  36.297562  31.530969   
Tetanus       LG-MG  7591   12.662238   6.111595  26.731976  21.388810   
                     7592    6.245053   1.909118  28.948171  24.097474   
                     8592   12.492029   5.017000  26.501067  21.990867   
                     8595    9.677529   3.982424  35.704529  29.460259   
              TA     7592    9.506387   3.153742  29.278355  24.325290   
                     8592    6.814328   2.068967  29.415508  23.994459   
                     8594   10.604713   4.549317  28.624307  23.662030   
                     8595   11.486895   4.823974  30.024513  25.416171   
Tetanus-Crush LG-MG  125    15.703600   6.478425  30.754475  26.679400   
                     135    10.172375   3.822875  34.323813  29.788469   
                     151    14.297944   9.260535  34.362648  29.458465   
                     152    13.325581   5.714372  30.776953  26.286140   
              TA     123    11.030691   3.895968  29.474043  24.041670   
                     135    13.557368   5.433930  36.171211  29.255719   
                     144     9.204835   2.791367  29.143063  23.764443   
                     151    19.341200   6.603543  31.287143  25.915029   
                     152    16.412338   6.599577  34.328014  27.903394   

                               marea1     marea2     vcov11     vcov12  \
Group         Muscle Mouse                                               
Crush         LG-MG  7482   15.654238   6.412238  44.728571  55.780952   
                     7485    5.997296   1.694481  27.703704  32.748148   
                     7488   19.212800   8.383100  52.107500  61.180000   
                     8831   18.337987  10.853785  67.817722  77.477215   
                     8834   15.422